# Learning With Errors (LWE)

LWE is the encryption-side counterpart to SIS. Introduced by Regev in 2005, it's the foundation for most lattice-based encryption schemes.

The basic idea: you get samples (a, b) where b = <a, s> + e (mod q). The secret s is hidden by small random errors e. Your job: recover s.

This is like solving a system of linear equations, but with noise added to the right-hand side. The noise makes it hard!

Credits: Alfred Menezes - [Lattice-Based Cryptography Course](https://www.youtube.com/playlist?list=PLA1qgQLL41STNFDvPJRqrHtuz0PIEJ4a8)

## What is LWE?

**LWE(m, n, q, B)**: Given A ∈ Z_q^(m×n) and b = As + e (mod q), find the secret s.

Parameters:
- s ∈ Z_q^n: the secret (what we're trying to find)
- A ∈ Z_q^(m×n): random public matrix (m samples)
- e ∈ [-B, B]^m: small error vector (noise)
- b = As + e (mod q): the noisy samples

Think of it as:
- Each row of A with corresponding entry in b gives one "sample" (a, b_i)
- Without errors (e=0), this is just linear algebra - easy to solve
- With errors, it's hard! The errors hide the secret s

Key constraint: B << q/2 (errors are small compared to modulus)

In [1]:
import numpy as np

# Simple LWE example: m=4, n=2, q=17, B=2
m, n, q, B = 4, 2, 17, 2

# Secret (what we're trying to find)
s = np.array([3, 5])

# Random matrix A
np.random.seed(42)
A = np.random.randint(0, q, size=(m, n))

# Small errors
e = np.array([1, -1, 0, 2])

# Compute b = As + e (mod q)
b = (A @ s + e) % q

print("LWE Example")
print("=" * 50)
print(f"Parameters: m={m}, n={n}, q={q}, B={B}\n")
print(f"Secret s = {s}  (this is what we want to find!)\n")
print(f"Matrix A:")
print(A)
print(f"\nErrors e = {e}  (small, in [-{B}, {B}])\n")
print(f"Public info b = As + e mod {q}:")
print(b)

# Verify
print(f"\nVerification:")
print(f"  As mod {q} = {(A @ s) % q}")
print(f"  As + e mod {q} = {(A @ s + e) % q}")
print(f"  Matches b? {np.array_equal((A @ s + e) % q, b)}")

print(f"\nThe problem: Given only A and b, recover s!")
print(f"Without errors, this is easy (linear algebra)")
print(f"With errors, it's believed to be hard!")

LWE Example
Parameters: m=4, n=2, q=17, B=2

Secret s = [3 5]  (this is what we want to find!)

Matrix A:
[[ 6 14]
 [10  7]
 [ 6 10]
 [10  3]]

Errors e = [ 1 -1  0  2]  (small, in [-2, 2])

Public info b = As + e mod 17:
[ 4 13  0 13]

Verification:
  As mod 17 = [ 3 14  0 11]
  As + e mod 17 = [ 4 13  0 13]
  Matches b? True

The problem: Given only A and b, recover s!
Without errors, this is easy (linear algebra)
With errors, it's believed to be hard!


## Multiple Solutions

One tricky thing about LWE: there might be multiple valid solutions!

From the classic example with m=5, n=3, q=31, B=2, there are actually 3 different (s, e) pairs that satisfy As + e = b (mod 31).

This is why we need:
- m >> n (many more samples than unknowns)
- Careful parameter selection to make multiple solutions very unlikely

In practice, with proper parameters, the probability of multiple solutions is negligible.

In [2]:
# Example with multiple solutions (from the classic example)
q = 31
A = np.array([
    [11, 3, 27],
    [12, 21, 7],
    [6, 23, 30],
    [5, 6, 2],
    [21, 0, 14]
])
b = np.array([25, 25, 12, 29, 17])

# Three different solutions
solutions = [
    (np.array([2, 11, 7]), np.array([-2, 0, 2, 1, 1])),
    (np.array([27, 13, 16]), np.array([1, -2, 1, 1, 1])),
    (np.array([30, 9, 5]), np.array([-2, -1, 2, 1, -1]))
]

print("Example with Multiple Solutions")
print("=" * 70)
print(f"Parameters: m=5, n=3, q={q}, B=2\n")
print("Matrix A (5×3) and target b:")
print(A)
print(f"b = {b}\n")

print("Found 3 different LWE solutions:\n")
for i, (s, e) in enumerate(solutions, 1):
    result = (A @ s + e) % q
    match = np.array_equal(result, b)
    is_short = np.all(np.abs(e) <= 2)
    
    print(f"Solution {i}:")
    print(f"  s = {s}")
    print(f"  e = {e}")
    print(f"  As + e mod {q} = {result}")
    print(f"  Matches b? {match}, Errors short? {is_short}")
    print()

print("This is why parameter selection matters!")
print("Need m >> n to make multiple solutions unlikely")

Example with Multiple Solutions
Parameters: m=5, n=3, q=31, B=2

Matrix A (5×3) and target b:
[[11  3 27]
 [12 21  7]
 [ 6 23 30]
 [ 5  6  2]
 [21  0 14]]
b = [25 25 12 29 17]

Found 3 different LWE solutions:

Solution 1:
  s = [ 2 11  7]
  e = [-2  0  2  1  1]
  As + e mod 31 = [25 25 12 29 17]
  Matches b? True, Errors short? True

Solution 2:
  s = [27 13 16]
  e = [ 1 -2  1  1  1]
  As + e mod 31 = [25 25 12 29 17]
  Matches b? True, Errors short? True

Solution 3:
  s = [30  9  5]
  e = [-2 -1  2  1 -1]
  As + e mod 31 = [25 25 12 29 17]
  Matches b? True, Errors short? True

This is why parameter selection matters!
Need m >> n to make multiple solutions unlikely


## Choosing B (Error Size)

The error bound B is critical:

**Too small (B=0)**: No errors! Just solve As = b, easy with linear algebra.

**Too large (B = q/2)**: Errors are as big as the modulus! Now b looks completely random and recovering s is information-theoretically impossible.

**Just right (B < q/4)**: Errors are small enough that they don't completely hide s, but large enough to make the problem hard.

There's also the Arora-Ge result: if B < sqrt(n), LWE can be solved in subexponential time. So we want B somewhere between sqrt(n) and q/4.

In practice: B is often in the range [1, 10] or so.

In [3]:
# Demonstrating different error sizes
q = 31
n = 3
s = np.array([5, 7, 11])

A = np.array([
    [3, 5, 7],
    [2, 8, 4],
    [9, 1, 6]
])

print("Effect of Error Size B")
print("=" * 70)
print(f"q = {q}, n = {n}, s = {s}\n")

# B = 0: No errors
e0 = np.array([0, 0, 0])
b0 = (A @ s + e0) % q
print(f"B = 0 (no errors):")
print(f"  e = {e0}")
print(f"  b = As mod {q} = {b0}")
print(f"  Easy to solve: Just compute A^(-1) b (if A invertible)")
print()

# B = 2: Small errors (good)
e2 = np.array([1, -2, 1])
b2 = (A @ s + e2) % q
print(f"B = 2 (small errors - typical):")
print(f"  e = {e2}")
print(f"  b = As + e mod {q} = {b2}")
print(f"  Hard to solve: Errors hide s, but not completely")
print()

# B = 15: Large errors (too big)
e15 = np.array([14, -15, 13])
b15 = (A @ s + e15) % q
print(f"B = 15 (large errors - too big!):")
print(f"  e = {e15}")
print(f"  b = As + e mod {q} = {b15}")
print(f"  Information-theoretically hard: b looks nearly random")
print()

print("Sweet spot: B < q/4, but B > sqrt(n)")
print(f"For q={q}, n={n}: sqrt(n) = {np.sqrt(n):.1f}, q/4 = {q/4:.1f}")
print(f"Good range: B ∈ [2, 7]")

Effect of Error Size B
q = 31, n = 3, s = [ 5  7 11]

B = 0 (no errors):
  e = [0 0 0]
  b = As mod 31 = [ 3 17 25]
  Easy to solve: Just compute A^(-1) b (if A invertible)

B = 2 (small errors - typical):
  e = [ 1 -2  1]
  b = As + e mod 31 = [ 4 15 26]
  Hard to solve: Errors hide s, but not completely

B = 15 (large errors - too big!):
  e = [ 14 -15  13]
  b = As + e mod 31 = [17  2  7]
  Information-theoretically hard: b looks nearly random

Sweet spot: B < q/4, but B > sqrt(n)
For q=31, n=3: sqrt(n) = 1.7, q/4 = 7.8
Good range: B ∈ [2, 7]


## Decisional LWE (DLWE)

There's a decision version of LWE that's important for encryption:

**DLWE**: Given (A, c), decide if:
- c = As + e (an LWE sample), OR
- c = random

This is like distinguishing "noisy linear combinations" from "pure noise".

**Key fact**: LWE and DLWE are equivalent in difficulty. If you can solve one, you can solve the other.

DLWE is what makes LWE-based encryption secure: ciphertexts look indistinguishable from random!

In [4]:
# DLWE example: can you tell which is which?
q = 31
n = 3
s = np.array([5, 7, 11])
A = np.array([
    [3, 5, 7],
    [2, 8, 4]
])

# LWE sample: c = As + e
e = np.array([1, -2])
c_lwe = (A @ s + e) % q

# Random sample
np.random.seed(123)
c_random = np.random.randint(0, q, size=2)

print("Decisional LWE Challenge")
print("=" * 70)
print("I give you matrix A and two vectors c1, c2")
print("One is an LWE sample (As + e), one is random")
print("Can you tell which is which?\n")

print(f"Matrix A:")
print(A)
print()

# Shuffle them
samples = [("Sample 1", c_lwe), ("Sample 2", c_random)]
import random
random.shuffle(samples)

for name, c in samples:
    print(f"{name}: {c}")
print()

print("Without knowing s, both look pretty random!")
print("This is why DLWE is hard and useful for encryption.")
print()
print("Answer: Sample 1 was", "LWE" if samples[0][1] is c_lwe else "random")
print("        Sample 2 was", "LWE" if samples[1][1] is c_lwe else "random")

Decisional LWE Challenge
I give you matrix A and two vectors c1, c2
One is an LWE sample (As + e), one is random
Can you tell which is which?

Matrix A:
[[3 5 7]
 [2 8 4]]

Sample 2: [30 13]
Sample 1: [ 4 15]

Without knowing s, both look pretty random!
This is why DLWE is hard and useful for encryption.

Answer: Sample 1 was random
        Sample 2 was LWE


## Short-Secret LWE (ss-LWE)

In standard LWE, the secret s is uniform in Z_q^n (can be any value mod q).

In short-secret LWE, the secret is also short: s ∈ [-B, B]^n (same bound as errors!).

**ss-LWE(m, n, q, B)**: Given A and b = As + e where both s and e are in [-B, B], find s.

Why care?
- More efficient: shorter secrets mean smaller keys
- Still hard: ss-LWE and LWE are equivalent!
- Used in practice: most real schemes use short secrets

The equivalence means: solving one version lets you solve the other.

In [5]:
# Comparing regular LWE vs short-secret LWE
q = 31
n = 3
B = 2

# Regular LWE: s can be anything mod q
s_regular = np.array([25, 17, 29])  # Large values mod q

# Short-secret LWE: s is also small
s_short = np.array([2, -1, 1])  # Small values in [-B, B]

A = np.array([
    [3, 5, 7],
    [2, 8, 4],
    [9, 1, 6],
    [7, 3, 2]
])

e = np.array([1, -2, 0, 1])

print("Regular LWE vs Short-Secret LWE")
print("=" * 70)
print(f"Parameters: n={n}, q={q}, B={B}\n")

print("Regular LWE:")
print(f"  Secret s = {s_regular}  (any values mod {q})")
b_regular = (A @ s_regular + e) % q
print(f"  b = As + e mod {q} = {b_regular}")
print()

print("Short-Secret LWE:")
print(f"  Secret s = {s_short}  (small, in [-{B}, {B}])")
b_short = (A @ s_short + e) % q
print(f"  b = As + e mod {q} = {b_short}")
print()

print("Benefits of short secrets:")
print("  - More efficient (smaller keys)")
print("  - Same hardness (equivalence proven)")
print("  - Used in practice (Kyber, Dilithium, etc.)")
print()
print(f"Storage comparison:")
print(f"  Regular secret: {n} elements, each ≤ {q} ≈ {n * np.log2(q):.1f} bits")
print(f"  Short secret: {n} elements, each ≤ {B} ≈ {n * np.log2(2*B+1):.1f} bits")

Regular LWE vs Short-Secret LWE
Parameters: n=3, q=31, B=2

Regular LWE:
  Secret s = [25 17 29]  (any values mod 31)
  b = As + e mod 31 = [23 21 13  6]

Short-Secret LWE:
  Secret s = [ 2 -1  1]  (small, in [-2, 2])
  b = As + e mod 31 = [ 9 29 23 14]

Benefits of short secrets:
  - More efficient (smaller keys)
  - Same hardness (equivalence proven)
  - Used in practice (Kyber, Dilithium, etc.)

Storage comparison:
  Regular secret: 3 elements, each ≤ 31 ≈ 14.9 bits
  Short secret: 3 elements, each ≤ 2 ≈ 7.0 bits


## Lindner-Peikert PKE: Encryption from LWE

Now the fun part: let's build a public-key encryption scheme from LWE!

**Key Generation**:
1. Alice picks short secret s ∈ [-B, B]^n and error e ∈ [-B, B]^n
2. Alice picks random matrix A ∈ Z_q^(n×n)
3. Alice computes b = As + e (mod q)
4. Public key: (A, b)
5. Secret key: s

**Security**: Learning anything about s from (A, b) is exactly the ss-DLWE problem!

The public key looks like an LWE instance - if you can break it, you can solve LWE.

In [6]:
# Lindner-Peikert Key Generation
q = 229
n = 3
B = 2

# Alice generates keys
np.random.seed(42)

# Secret and error
s = np.array([2, -2, 1])
e = np.array([0, -2, 1])

# Random matrix
A = np.array([
    [101, 173, 27],
    [192, 121, 7],
    [116, 223, 30]
])

# Public key component
b = (A @ s + e) % q

print("Lindner-Peikert Key Generation")
print("=" * 70)
print(f"Parameters: n={n}, q={q}, B={B}\n")

print("Alice's secret key:")
print(f"  s = {s}  (short, in [-{B}, {B}])")
print(f"  e = {e}  (small error)")
print()

print("Alice generates public key:")
print(f"  Matrix A (random):")
print(A)
print(f"\n  b = As + e mod {q}:")
print(f"  b = {b}")
print()

print("Public key: (A, b)")
print("Secret key: s")
print()
print("Security: Anyone trying to learn s from (A, b)")
print("         must solve the ss-LWE problem!")

Lindner-Peikert Key Generation
Parameters: n=3, q=229, B=2

Alice's secret key:
  s = [ 2 -2  1]  (short, in [-2, 2])
  e = [ 0 -2  1]  (small error)

Alice generates public key:
  Matrix A (random):
[[101 173  27]
 [192 121   7]
 [116 223  30]]

  b = As + e mod 229:
  b = [112 147  46]

Public key: (A, b)
Secret key: s

Security: Anyone trying to learn s from (A, b)
         must solve the ss-LWE problem!


## Encryption

To encrypt a message bit m ∈ {0, 1}:

**Bob does**:
1. Get Alice's public key (A, b)
2. Pick random short vectors: r, z ∈ [-B, B]^n and z' ∈ [-B, B]
3. Compute:
   - c1 = A^T·r + z (mod q)
   - c2 = b^T·r + z' + m·⌊q/2⌋ (mod q)
4. Send ciphertext c = (c1, c2)

The key idea:
- c1 and c2 are LWE samples (look random due to z, z')
- The message m is hidden at the "most significant bit" by adding m·⌊q/2⌋
- If m=1, we add roughly q/2; if m=0, we add 0

This hides m in the noise!

In [7]:
# Encryption
m = 1  # Message bit to encrypt

# Bob's randomness
r = np.array([2, -2, -1])
z = np.array([0, 1, -2])
z_prime = -2

# Encrypt
c1 = (A.T @ r + z) % q
c2 = (b @ r + z_prime + m * (q // 2)) % q

print("Lindner-Peikert Encryption")
print("=" * 70)
print(f"Message: m = {m}\n")

print("Bob's randomness:")
print(f"  r = {r}")
print(f"  z = {z}")
print(f"  z' = {z_prime}")
print()

print("Encryption:")
print(f"  c1 = A^T·r + z mod {q}")
print(f"     = {c1}")
print(f"\n  c2 = b^T·r + z' + {m}·{q//2} mod {q}")
print(f"     = {c2}")
print()

print(f"Ciphertext: c = (c1, c2) = ({c1}, {c2})")
print()
print("Key idea: m is hidden at 'most significant bit' position")
print(f"  m=0: add 0")
print(f"  m=1: add {q//2} ≈ q/2")
print("The random terms z, z', r make it look random!")

Lindner-Peikert Encryption
Message: m = 1

Bob's randomness:
  r = [ 2 -2 -1]
  z = [ 0  1 -2]
  z' = -2

Encryption:
  c1 = A^T·r + z mod 229
     = [160 111   8]

  c2 = b^T·r + z' + 1·114 mod 229
     = 225

Ciphertext: c = (c1, c2) = ([160 111   8], 225)

Key idea: m is hidden at 'most significant bit' position
  m=0: add 0
  m=1: add 114 ≈ q/2
The random terms z, z', r make it look random!


## Decryption

To decrypt ciphertext c = (c1, c2):

**Alice does**:
1. Compute c2 - s^T·c1 (mod q)
2. Apply Round_q to recover m

**Why this works**:
```
c2 - s^T·c1 = (b^T·r + z' + m·⌊q/2⌋) - s^T·(A^T·r + z)
            = (s^T·A^T + e^T)·r + z' + m·⌊q/2⌋ - s^T·A^T·r - s^T·z
            = e^T·r - s^T·z + z' + m·⌊q/2⌋
```

The term e^T·r - s^T·z + z' is small noise (all terms are bounded by B).

So we get: (small noise) + m·⌊q/2⌋

**Rounding**: If result is close to 0, output m=0. If close to q/2, output m=1.

As long as the noise is < q/4, we correctly recover m!

In [8]:
# Decryption
def round_q(x, q):
    """Round to nearest message bit"""
    # Convert to centered representation
    x_centered = x if x <= q//2 else x - q
    # Round: if closer to q/2 than to 0, output 1
    if -q/4 < x_centered < q/4:
        return 0
    else:
        return 1

# Alice decrypts
val = (c2 - s @ c1) % q
m_recovered = round_q(val, q)

print("Lindner-Peikert Decryption")
print("=" * 70)
print(f"Ciphertext: c1 = {c1}, c2 = {c2}\n")

print("Alice uses secret key s:")
print(f"  s = {s}")
print()

print("Compute c2 - s^T·c1:")
print(f"  s^T·c1 = {(s @ c1) % q}")
print(f"  c2 - s^T·c1 mod {q} = {val}")
print()

# Show in centered form
val_centered = val if val <= q//2 else val - q
print(f"In centered form: {val_centered}")
print(f"Expected for m={m}: ≈ {m * (q//2)}")
print()

print(f"Apply Round_{q}:")
print(f"  -q/4 = {-q/4:.1f}, q/4 = {q/4:.1f}")
print(f"  {val_centered} is ", end="")
if -q/4 < val_centered < q/4:
    print(f"in [-q/4, q/4] → m = 0")
else:
    print(f"outside [-q/4, q/4] → m = 1")
print()

print(f"Recovered message: m = {m_recovered}")
print(f"Original message:  m = {m}")
print(f"{'✓ Success!' if m_recovered == m else '✗ Failure'}")

Lindner-Peikert Decryption
Ciphertext: c1 = [160 111   8], c2 = 225

Alice uses secret key s:
  s = [ 2 -2  1]

Compute c2 - s^T·c1:
  s^T·c1 = 106
  c2 - s^T·c1 mod 229 = 119

In centered form: -110
Expected for m=1: ≈ 114

Apply Round_229:
  -q/4 = -57.2, q/4 = 57.2
  -110 is outside [-q/4, q/4] → m = 1

Recovered message: m = 1
Original message:  m = 1
✓ Success!


## Complete Example: Encrypt Both Bits

Let's encrypt both m=0 and m=1 to see the full scheme in action.

In [9]:
# Complete encryption/decryption for both message bits
print("Complete Lindner-Peikert PKE Example")
print("=" * 70)
print(f"Parameters: n={n}, q={q}, B={B}\n")

for test_m in [0, 1]:
    print(f"\n{'='*70}")
    print(f"Testing message m = {test_m}")
    print('='*70)
    
    # Fresh randomness for each encryption
    r_test = np.random.randint(-B, B+1, size=n)
    z_test = np.random.randint(-B, B+1, size=n)
    zp_test = np.random.randint(-B, B+1)
    
    # Encrypt
    c1_test = (A.T @ r_test + z_test) % q
    c2_test = (b @ r_test + zp_test + test_m * (q // 2)) % q
    
    print(f"\nEncryption:")
    print(f"  Random: r={r_test}, z={z_test}, z'={zp_test}")
    print(f"  c1 = {c1_test}")
    print(f"  c2 = {c2_test}")
    
    # Decrypt
    val_test = (c2_test - s @ c1_test) % q
    val_centered_test = val_test if val_test <= q//2 else val_test - q
    m_recovered_test = round_q(val_test, q)
    
    print(f"\nDecryption:")
    print(f"  c2 - s^T·c1 mod {q} = {val_test}")
    print(f"  Centered: {val_centered_test}")
    print(f"  Expected for m={test_m}: ≈ {test_m * (q//2)}")
    print(f"  Recovered: m = {m_recovered_test}")
    print(f"  {'✓ Correct!' if m_recovered_test == test_m else '✗ Wrong!'}")

print("\n" + "=" * 70)
print("Lindner-Peikert PKE works!")
print("\nSecurity: Based on hardness of ss-DLWE")
print("- Ciphertexts look random (indistinguishable from uniform)")
print("- Breaking the scheme requires solving LWE")

Complete Lindner-Peikert PKE Example
Parameters: n=3, q=229, B=2


Testing message m = 0

Encryption:
  Random: r=[1 2 0], z=[ 2  2 -1], z'=0
  c1 = [ 29 188  40]
  c2 = 177

Decryption:
  c2 - s^T·c1 mod 229 = 226
  Centered: -3
  Expected for m=0: ≈ 0
  Recovered: m = 0
  ✓ Correct!

Testing message m = 1

Encryption:
  Random: r=[0 0 2], z=[1 0 2], z'=-1
  c1 = [  4 217  62]
  c2 = 205

Decryption:
  c2 - s^T·c1 mod 229 = 111
  Centered: 111
  Expected for m=1: ≈ 114
  Recovered: m = 1
  ✓ Correct!

Lindner-Peikert PKE works!

Security: Based on hardness of ss-DLWE
- Ciphertexts look random (indistinguishable from uniform)
- Breaking the scheme requires solving LWE


## Summary

**LWE (Learning With Errors)**: Given A and b = As + e (mod q), find secret s.
- Parameters: m (samples), n (dimension), q (modulus), B (error bound)
- Key constraint: B << q/2 (errors small but not too small)
- Security: Based on lattice problems (believed quantum-resistant!)

**Variants**:
- **DLWE**: Distinguish LWE samples from random (equivalent to LWE)
- **ss-LWE**: Secret s is also short (equivalent to LWE, more efficient)

**Applications**:
- **Public-key encryption**: Lindner-Peikert scheme (and many others)
- Message hidden at "most significant bit" position
- Security from ss-DLWE hardness
- Ciphertexts look random

**Parameter Selection**:
- m >> n for uniqueness
- B in [sqrt(n), q/4] for security
- Need B^2 << q/4 for correct decryption
- Typical: n=256-1024, q=3329-8380417, B=2-3

**Why LWE is Important**:
- Foundation for lattice-based encryption (Kyber, Frodo, etc.)
- Quantum-resistant (no known quantum attacks)
- Efficient implementations possible
- Ring-LWE and Module-LWE variants even more efficient